# Experiments with backtesting.py
This notebook is to explore the `backtesting.py` to evaluate strategy:
1. Based on conventional TA (talib etc.)
2. See how can we introduce ML model as TA

Install ta-lib on Mac with brew first `brew install ta-lib` and then `pip install TA-Lib`

In [13]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import yfinance as yf
import talib 
from bt_utils import get_stock_data

### 1. Standard Examples
First, use standard example; where goal is just to see how we can use the library. In the following, we will use SMA as an indicator which is already available in `backtesting.py`. We will also use stop loss and take profit to limit the exposure and to ensure risk-reward ratio.

In [14]:
class SmaCrossWithSLTP(Strategy):
    short_window = 10  # Fast SMA
    long_window = 20   # Slow SMA
    stop_loss_pct = 0.02   # 2% Stop Loss
    take_profit_pct = 0.04  # 4% Take Profit
    
    # All the necessary indicator needs to define in the init method
    def init(self):
        self.sma_short = self.I(SMA, self.data.Close, self.short_window)
        self.sma_long = self.I(SMA, self.data.Close, self.long_window)
    
    # It defines the logic of the strategy
    def next(self):
        price = self.data.Close[-1]  # Current price
        if crossover(self.sma_short, self.sma_long):  # Buy condition
            stop_loss = price * (1 - self.stop_loss_pct)
            take_profit = price * (1 + self.take_profit_pct)
            self.buy(sl=stop_loss, tp=take_profit)  # Order with SL and TP
        
        elif crossover(self.sma_long, self.sma_short):  # Sell condition
            self.position.close()  # Close existing position

# Load sample data which comes with the library to test
df = GOOG

# Run Backtest
bt = Backtest(df, SmaCrossWithSLTP, cash=10000, commission=0.002)
stats = bt.run()
print(stats)
# bt.plot()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     9.82309
Equity Final [$]                  11532.01791
Equity Peak [$]                   12667.87271
Commissions [$]                    2041.36509
Return [%]                           15.32018
Buy & Hold Return [%]               607.37036
Return (Ann.) [%]                     1.68634
Volatility (Ann.) [%]                 6.32304
CAGR [%]                              1.15945
Sharpe Ratio                           0.2667
Sortino Ratio                         0.40031
Calmar Ratio                          0.14875
Alpha [%]                            -4.51391
Beta                                  0.03266
Max. Drawdown [%]                   -11.33648
Avg. Drawdown [%]                    -3.53535
Max. Drawdown Duration      742 days 00:00:00
Avg. Drawdown Duration      172 days 00:00:00
# Trades                          

## 2. Integrating other indicators from TA-LIB
Now, moving to `ta-lib` for more indicators. In the following, we will use RSI along with SL and TP.

In [15]:
class RsiSmaStrategy(Strategy):
    rsi_period = 14    # RSI period
    rsi_buy = 30       # RSI oversold threshold
    rsi_sell = 70      # RSI overbought threshold
    stop_loss_pct = 0.02  # 2% stop-loss
    take_profit_pct = 0.05  # 5% take-profit

    def init(self):    
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_period)

    def next(self):
        # Entry Condition: RSI below 30 & SMA crossover
        if self.rsi[-1] < self.rsi_buy :
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  # Stop-Loss 2% below entry
            tp = price * (1 + self.take_profit_pct)  # Take-Profit 5% above entry
            self.buy(sl=sl, tp=tp)  # Place order with SL & TP

        # Exit Condition: RSI above 70 OR SMA crossover in opposite direction
        elif self.rsi[-1] > self.rsi_sell:
            self.position.close()

df = get_stock_data("AAPL")

# Run Backtest
bt = Backtest(df, RsiSmaStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
# Save HTML file for plots for in-browser visualization
bt.plot(filename="rsi_strategy.html")

[*********************100%***********************]  1 of 1 completed
Backtest.run:   0%|          | 0/527 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=17: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=403: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=409: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=505: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Start                     2022-01-03 00:00:00
End                       2024-02-29 00:00:00
Duration                    787 days 00:00:00
Exposure Time [%]                     7.01107
Equity Final [$]                  11851.73611
Equity Peak [$]                   11851.73611
Commissions [$]                     298.72019
Return [%]                           18.51736
Buy & Hold Return [%]                13.25856
Return (Ann.) [%]                     8.21925
Volatility (Ann.) [%]                 8.25065
CAGR [%]                              5.59059
Sharpe Ratio                           0.9962
Sortino Ratio                         2.07463
Calmar Ratio                           2.1929
Alpha [%]                            17.55187
Beta                                  0.07282
Max. Drawdown [%]                    -3.74812
Avg. Drawdown [%]                    -2.09708
Max. Drawdown Duration       16 days 00:00:00
Avg. Drawdown Duration        7 days 00:00:00
# Trades                          

GridPlot(id='p3233', ...)

## 3. Integrating a black-box indicator (e.g., ML model)
Now we will add a black-box function which can be an ML model which can provide BUY or SELL signal. We will pass the dataframe containing the features and function can use this to predict the position. This then can be used in `next` function.

In [16]:
def dummy_ml_model(df):
    """
    Simulated ML model that predicts BUY signals. We don't consider SELL (short positions).
    :param df: DataFrame of last `N` rows (OHLCV features).
    :return: 1 if BUY signal, else 0.
    """
    # Simulated logic: Buy if the last row closes lower than the first row in the window
    if df.iloc[-1]['Close'] < df.iloc[0]['Close']:  
        return 1  # BUY Signal
    return 0  # No action

class MLTradingStrategy(Strategy):
    feature_window = 5 # window size for features 
    stop_loss_pct = 0.02 
    take_profit_pct = 0.05 

    def init(self):
        pass

    def next(self):
        N = self.feature_window  
        # Ensure enough data to generate features
        if len(self.data.Close) < N:
            return 

        # Extract last `N` rows into a DataFrame
        df = pd.DataFrame({
            "Open": self.data.Open[-N:],
            "High": self.data.High[-N:],
            "Low": self.data.Low[-N:],
            "Close": self.data.Close[-N:],
            "Volume": self.data.Volume[-N:],
        })

        # Pass last `N` rows to ML model
        signal = dummy_ml_model(df)  

        # ML suggests BUY signal
        if signal == 1:  
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  
            tp = price * (1 + self.take_profit_pct) 
            self.buy(sl=sl, tp=tp)  

df = get_stock_data("AAPL") 

# Run Backtest
bt = Backtest(df, MLTradingStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
bt.plot(filename="ml__strategy.html")

[*********************100%***********************]  1 of 1 completed
Backtest.run:   0%|          | 0/541 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=17: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=25: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=26: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=30: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/si

Start                     2022-01-03 00:00:00
End                       2024-02-29 00:00:00
Duration                    787 days 00:00:00
Exposure Time [%]                     70.1107
Equity Final [$]                   4967.12098
Equity Peak [$]                   10102.65372
Commissions [$]                    3503.94424
Return [%]                          -50.32879
Buy & Hold Return [%]                 0.57056
Return (Ann.) [%]                   -27.77201
Volatility (Ann.) [%]                17.77616
CAGR [%]                            -20.07333
Sharpe Ratio                         -1.56232
Sortino Ratio                         -1.4636
Calmar Ratio                         -0.51764
Alpha [%]                           -50.67527
Beta                                  0.60726
Max. Drawdown [%]                   -53.65079
Avg. Drawdown [%]                   -27.12878
Max. Drawdown Duration      778 days 00:00:00
Avg. Drawdown Duration      391 days 00:00:00
# Trades                          

GridPlot(id='p3587', ...)